In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/KPDL/LSTM

/content/drive/My Drive/KPDL/LSTM


In [0]:
ll

total 67385
-rw------- 1 root 63461217 Nov 10 03:22 embedding_matrix.npz
-rw------- 1 root     5454 Nov 10 03:24 Model1_LSTM.ipynb
-rw------- 1 root  1010048 Nov 10 03:24 test_data.pkl
-rw------- 1 root  2427710 Nov 10 03:22 tokenizer_60971_token.pickle
-rw------- 1 root  2097152 Nov 10 03:24 train_data.pkl


**Preprocess, Tokenizer**

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import MWETokenizer, word_tokenize, RegexpTokenizer
import re
import nltk
import unicodedata

multiple_punctuation_pattern = re.compile(r"([\"\.\?\!\,\:\;\-])(?:[\"\.\?\!\,\:\;\-]){1,}")
word_tokenizer = MWETokenizer(separator='')
multiple_emoji_pattern = re.compile(u"(["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u00a9"
        u"\u00ae"
        u"\u2000-\u3300"
        "]){1,}", flags= re.UNICODE )

normalizer = {'òa': 'oà',
              'óa': 'oá',
              'ỏa': 'oả',
              'õa': 'oã',
              'ọa': 'oạ',
              'òe': 'oè',
              'óe': 'oé',
              'ỏe': 'oẻ',
              'õe': 'oẽ',
              'ọe': 'oẹ',
              'ùy': 'uỳ',
              'úy': 'uý',
              'ủy': 'uỷ',
              'ũy': 'uỹ',
              'ụy': 'uỵ',
              'Ủy': 'Uỷ'}
correct_mapping = {
      "m": "mình",
      "mik": "mình",
      "ko": "không",
      "k": " không ",
      "kh": "không",
      "khong": "không",
      "kg": "không",
      "khg": "không",
      "tl": "trả lời",
      "r": "rồi",
      "ok": "tốt",
      "dc": "được",
      "vs": "với",
      "đt": "điện thoại",
      "thjk": "thích",
      "thik": "thích",
      "qá": "quá",
      "trể": "trễ",
      "bgjo": "bao giờ",
      "''": '"',
      "``": '"'
}

def normalize_text(text):
  for absurd, normal in normalizer.items():
    text = text.replace(absurd, normal)

  # for l in vn_location:
  #   text = text.replace(l, ' location ')

  return text

def tokmap(tok):
  if tok.lower() in correct_mapping:
      return correct_mapping[tok.lower()]
  else:
      return tok

def preprocess(text):
  text = text.lower()
  text = multiple_emoji_pattern.sub(r"\g<1> ", text) # \g<1>
  text = multiple_punctuation_pattern.sub(r" \g<1> ", text)
  text = unicodedata.normalize("NFC", text)
  text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b(\/)?', 'url', text)
  text = re.sub("\.", " . ", text)
  text = re.sub("'", "' ", text)
  text = re.sub('"', '" ', text)
  text = re.sub('/', ' / ', text)
  text = re.sub('-', ' - ', text)
  text = re.sub(',', ' , ', text)
  text = re.sub(r'\s{2,}', ' ', text)
  text = normalize_text(text)
  # text = re.sub(r'\#[^\s]+', ' hastag ', text)
  text = re.sub(r'(|\s)([\d]+k)(\s|$)', ' cureency_k ', text)
  text = re.sub(r'(([\d]{2,4}\s){2,}([\d]+)?|(09|01|[2|6|8|9]|03)+([0-9]{8})\b)', ' phone_number ', text)
  # text = re.sub(r'\d', "_digit", text)
  tokens = word_tokenizer.tokenize(word_tokenize(text))
  tokens = list(map(tokmap, tokens))
  # return tokens
  return ' '.join(tokens)

print(preprocess('Địa chỉ : 103 d9 ngõ 63 , thái thịnh, đống đa, hà nội.'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
địa chỉ : 103 d9 ngõ 63 , thái thịnh , đống đa , hà nội .


**Load training data**

In [0]:
from sklearn.model_selection import train_test_split

with open('./sell_detection_train.v1.0.txt') as f:
  xtrain = f.read().strip().split('\n')
  ytrain = [ line.split(' ',1)[0] for line in xtrain]
  xtrain = [ line.split(' ',1)[1] for line in xtrain]
  xtrain = [ preprocess(descriptions) for descriptions in xtrain]
print(len(xtrain))
print(xtrain[:10])
print(ytrain[:10])

60000
['gsu khsi nek dep gai hk cho y kirn yk nhoaz khsi huong nhiu', 'haiz . chan wa . hoc - hoc - nua - hoc mau', 'găng tay ngập shop', '🌺 2 siểu phẩm xông bắc thảo dược 🌺 ( công dụng từng loại em kèm dưới hình nha khách yêu ) 💧 xông bắc đặc trị mụn ( dùng cho khách nhiều mụn - trị được tất cả loại mụn ) 💧 xông bắc trắng da ( kềm nhờn - trắng mịn da - loại bỏ mụn đầu đen - mụn cám ) 💫 loại xông trắng khi xông hơi xong khách có thể dùng nước thảo dược để tắm trắng dưỡng da luôn ạ . em bán mát tay 2 loại này lắm luôn nha khách 💋 nam nữ sử dụng đều được à ⚡ ️⚡ ️⚡ ️hàng xông nhà em không pha tạp , đầy đủ thành phần , an toàn cho da vì là 100 % thiên nhiên : 3 à mà khách biết tánh em đó giờ luôn bán hàng thiên nhiên dòiii ❤ ️', 'nổi nhớ dân trào 😂', '" hoi trai sum vay "', 'thật sự jo này mình rất buồn . cố nhưng không thể', 'tổ quốc chưa bao giờ thiêng liêng đến thế .', '- chất liệu : da thật , lót da - màu : đen , mận - s35 - 39 form thường - giá : cureency_k - địa chỉ : số 1 ngõ 850 đư

In [0]:
def maping_lable(lable):
  if lable == '__label__post_khong_ban':
    return 0
  return 1
ytrain = list(map(maping_lable, ytrain))
print(ytrain[:10])

[0, 0, 0, 1, 0, 0, 0, 0, 1, 1]


In [0]:
train_data = pd.concat([train_data, test_data])
train_data = train_data.sample(frac=1).reset_index(drop=True)
print(len(train_data))

**Load embedding_matrix + tokenizer**

In [0]:
import numpy as np
import pickle

embedding_matrix = np.load('embedding_matrix_91104_tokens.npz')['embeddings']
print(type(embedding_matrix))
print(len(embedding_matrix))

with open("tokenizer_91104_tokens.pickle", "rb") as input_file:
    tokenizer = pickle.load(input_file)
    print(len(tokenizer.word_counts))

<class 'numpy.ndarray'>
91105
91103


**Model setting**

In [0]:
pip install Keras==2.3.1

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import Constant

print('number of words: ', len(tokenizer.word_counts))

list_tokenized_train = tokenizer.texts_to_sequences(xtrain) # data.x_train
list_tokenized_test = tokenizer.texts_to_sequences(xtrain[:10000]) # data.x_test

maxlen = 180
X_train = pad_sequences(list_tokenized_train , maxlen=maxlen, padding='post')
Y_train = ytrain #data.y_train #train_data['labels']

X_test = pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post')
Y_test = ytrain[:10000] #test_data['labels']
print('train-test:', len(X_train), len(X_test))



number of words:  91103
train-test: 60000 10000


In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, Sequential
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
# from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import Constant

embed_size = 300
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                    output_dim=embed_size,
                    embeddings_initializer=Constant(embedding_matrix),
                    trainable=True))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
# mcp_save = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True, monitor='val_acc', mode='max')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='max')

batch_size = 1024
epochs = 4
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs )
          #validation_data=(X_test, Y_test) ) #, callbacks=[mcp_save]) # validation_split=0.1, validation_data=(X_test, Y_test)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         27331500  
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 64)          85248     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
______________________________________

**Load test data**

In [0]:
with open('./sell_detection_test.v1.0.txt') as f:
    sell_detection_test = f.read().strip().split('\n')
    sell_detection_test = [ preprocess(e) for e in sell_detection_test]
print(sell_detection_test[:10])
print(len(sell_detection_test))


['\ufeffbéo mà .', 'minh la tai nguyen', 'moi bat được con nay nhau được hok', 'bữa ăn sáng trên bầu trời', '❤ ️ 01 / 01 / 2013 - 01 / 01 / 2018 : gặp được nhau đã là cái duyên , đến với nhau , bên cạnh nhau , quan tâm nhau . có lẽ là do số phận ! 5 năm thanh xuân có nhau . lãi được 1 cục 💎 p / s : mỗi năm lãi đều như vậy thì lời to rồi 😍 # 5year_anniversary # htlovemn # kếhoạch5nămlần1', 'sơri võ', 'ngu thoj chuc ka nha ngu ngon', 'thực hiện chính sách đại lý mới nhằm chống bán phá giá để bảo vệ thương hiệu và các đại lý của istitch . kể từ nay istitch chỉ bán cho 1 đại lý ở mỗi tỉnh thành . các quý đại lý có nhu cầu xin vui lòng liên hệ email : thanhhongdo @ gmail . com để lấy báo giá riêng nhé ! xin cám ơn .', 'snowntown sài gòn ❤ ️⛄ ️❄ ️', 'ດົນໆໄປເທື່ອ . ມ່ວນຫຼາຍມື້ນີ້']
98893


In [0]:
list_tokenized_test = tokenizer.texts_to_sequences(sell_detection_test)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post')
print(len(X_test))

98893


**Generate prediction to file**

In [0]:
predicted = model.predict_classes(X_test)
print(sum(predicted))
def reverse_maping_lable(lable):
  if lable == 1:
    return '__label__post_ban_hang'
  return '__label__post_khong_ban'

predicted_origin_lables = list(map(reverse_maping_lable, predicted))
with open('sell_detection_team03_solution2.result.txt', 'w') as f:
  f.write('\n'.join(predicted_origin_lables))

[13839]


In [0]:
# y_pred = model.predict_classes(X_test)
# from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report
# print('F1-score: {0}'.format(f1_score(Y_test, y_pred)))
# print('precision_score: {0}'.format(precision_score(Y_test, y_pred)))
# print('recall_score: {0}'.format(recall_score(Y_test, y_pred)))
# print('\nConfusion matrix: \n{}\n'.format(confusion_matrix(Y_test, y_pred)))
# print(classification_report(Y_test, y_pred, digits=6))

F1-score: 0.9418472063854048
precision_score: 0.9315789473684211
recall_score: 0.9523443504996156

Confusion matrix: 
[[8608   91]
 [  62 1239]]

              precision    recall  f1-score   support

           0   0.992849  0.989539  0.991191      8699
           1   0.931579  0.952344  0.941847      1301

    accuracy                       0.984700     10000
   macro avg   0.962214  0.970942  0.966519     10000
weighted avg   0.984878  0.984700  0.984772     10000



In [0]:
with open('predict_lable.txt', 'w') as f:
  with open('../FastText Model/sell_detection_test.v1.0.txt') as f2:
    sell_detection_test_file = f2.read().strip().split('\n')
  for i in range(len(predicted_origin_lables)):
    f.write(predicted_origin_lables[i] + ' ' + sell_detection_test_file[i]+'\n')

In [0]:
with open('../FastText Model/sell_detection_team03_solution1.result.txt') as f:
  s1 = f.read().strip().split('\n')
with open('sell_detection_team03_solution2.1.result.txt') as f:
  s2 = f.read().strip().split('\n')
with open('compare_svm_vs_lstm.txt', 'w') as f:
  f.write('index, lablessvm , lableslstm2.1, test line\n')
  for i in range(len(sell_detection_test)):
    if s1[i] != s2[i]:
      f.write(str(i) + ' ' + s1[i] + ' ' + s2[i] + ' ' + sell_detection_test[i]+'\n')


In [0]:
with open('compare_lstm2.1_svm.txt') as f:
  print(len(f.read().split('\n')))

1627


In [0]:
print(sell_detection_test)